# Digital Signal Processing

### MsCV - ViBOT - MAIA

In [ ]:
%matplotlib inline
%pprint off

# To make some nice plot
import matplotlib.pyplot as plt
# To play with arrays
import numpy as np
import scipy as sp
from skimage import io

from skimage import img_as_float
from matplotlib import cm
from skimage import img_as_ubyte
from scipy import signal



/home/lemaitre/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


## Filtering 
**1.1**

**1.2**

## Recursive filter 

**2.1**

**2.2**

**2.3**

**2.4**

**3.1**

**3.2** , **3.3**

**3.4**